In [1]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from decimal import *

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [8]:
customer_id = '137'
customer_name='%kimballwest%'
buckets = "Fixings & Fasteners"
# strategy_version_input=705
# attribute_id_input=5131
formatted_attribute = 'burst_pressure_psi'

In [9]:
# parameters
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [10]:
import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
field='BurstPressure'
fields = [field]
# df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df], axis = 1)
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
print(len(df))

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']

import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

1027


In [11]:
# df[field].explode().value_counts()
df.head(0)

Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, BurstPressure]
Index: []

In [72]:
# start=r'(?i)(^.{0,10})|()'
# df['start']=df['product_name'].apply(lambda x: re_extract(start, str(x)))

pattern=r'''(P\W?S\W?I)|()''' 
df['name'] = df['product_name'].apply(lambda x: re_extract(pattern, str(x)))
df['ld'] = df['long_desc'].apply(lambda x: re_extract(pattern, str(x)))



pat=r'(\d+\,?\d+.{0,3}P\W?S\W?I)|()'
df['matches']=df[field].apply(lambda x: re_extract(pat, str(x)))

# pat=r'(?i)(m)|()'
# df['mm']=df[field].apply(lambda x: re_extract(pat, str(x)))


custom=df[(df[field].astype(str)!='nan')]
custom[curation_col]=custom[field].apply(lambda x: re.sub(r'(for.+)','',str(x))).apply(lambda x: re.sub(r"P\W?S\W?I\W?.*",'',str(x))).apply(lambda x: re.sub(r'(?i)(950\s+.\s+1\,400)','950 - 1,400',str(x))).apply(lambda x: re.sub(r'"','',str(x))).apply(lambda x: re.sub(r'(?i)Assortment|\s','',str(x))).apply(lambda x: re.sub(r'\-',' - ',str(x)))#.apply(lambda x: re.sub(r'\s?"\s?',' in',str(x))).apply(lambda x: f'{x} in').apply(lambda x: re.sub(r'\s?in\s?in',' in',str(x)))                                          
# custom[curation_col]=custom[field].apply(lambda x: f'{x}"').apply(lambda x: re.sub(r'""','"',str(x)))
print(len(custom))
matchcustom=custom[['external_id',curation_col]]
matchcustom[curation_col]=matchcustom[curation_col]
matchcustom=matchcustom[matchcustom[curation_col].astype(str)!='']
matchcustom[curation_col].explode().value_counts()

547


12,000         80
1,400          33
8,000          32
9,000          29
1,000          28
1,200          27
16,000         21
20,000         21
428            17
2,000          16
23,200         16
11,000         15
950            14
6,000          12
14,500         12
10,000         10
900             9
19,200          8
400             8
7,000           7
18,000          7
14,000          6
1,600           6
17,000          6
22,000          6
800             6
600             6
3,200           5
450             5
5,000           5
12,500          4
250             4
12,200          3
6,500           3
4,000           3
1,800           3
2,500           3
165             3
1,750           3
8,100           2
22,340          2
540             2
26,120          2
950 - 1,400     2
18,500          2
17,400          2
13,000          2
14,800          2
13,600          2
26,000          2
12,800          2
525             2
300             1
500             1
3,600           1
195       

In [64]:
not_match=df[(df[field].astype(str)=='nan')]
# print(len(not_match[(not_match['ld'].astype(str)=='[]')|(not_match['name'].astype(str)!='[]')]))
two=not_match[(not_match['ld'].astype(str)!='[]')|(not_match['name'].astype(str)!='[]')]
# print(len(two))
# two[curation_col]=two['name'].apply(lambda x: re.sub(r'\s?\-\s?',' x ',str(x))).apply(lambda x: re.sub(r"'",'"',str(x)))
# matchtwo=two[['external_id',curation_col]]
# print(len(matchtwo))
two

buckets external_id  \
23             Hoses       54657   
24             Hoses       54658   
25             Hoses       54659   
26             Hoses       54626   
27             Hoses       54660   
28             Hoses       54628   
29             Hoses       54627   
30             Hoses       54629   
34             Hoses       54216   
35             Hoses       54215   
36             Hoses       54214   
41             Hoses       54212   
42             Hoses       54211   
43             Hoses       54210   
314            Hoses       54710   
480            Hoses      541716   
485            Hoses      541717   
486            Hoses      541718   
487            Hoses      541711   
488            Hoses      541712   
489            Hoses      541713   
490            Hoses      541714   
491            Hoses      541715   
492            Hoses      541719   
493            Hoses      541720   
494            Hoses      541721   
495            Hoses      541722   
496            Hoses       54320   
497            Hoses       54321   
499            Hoses       54322   
500            Hoses       54323   
501            Hoses       54324   
502            Hoses       54325   
503            Hoses       54326   
504            Hoses       54327   
505            Hoses       54328   
506            Hoses       54329   
507            Hoses       54330   
514            Hoses       54922   
515            Hoses       54920   
516            Hoses       54924   
517            Hoses       54934   
518            Hoses       54930   
519            Hoses       54921   
520            Hoses       54925   
521            Hoses       54932   
523            Hoses       54923   
525            Hoses       54933   
530            Hoses       54931   
531            Hoses       54720   
532            Hoses       54618   
533            Hoses       54721   
534            Hoses       54624   
535            Hoses       54620   
536            Hoses       54726   
537            Hoses       54610   
538            Hoses       54616   
541            Hoses       54612   
542            Hoses       54611   
543            Hoses       54622   
553            Hoses       54643   
560            Hoses       54641   
562            Hoses       54648   
563            Hoses       54647   
564            Hoses       54642   
599            Hoses       54731   
600            Hoses       54730   
601            Hoses       54548   
602            Hoses       54547   
603            Hoses       54546   
607   Pipes & Tubing       60260   
608   Pipes & Tubing       60259   
609   Pipes & Tubing       60256   
610   Pipes & Tubing       60255   
611   Pipes & Tubing       60258   
612   Pipes & Tubing       60257   
801   Pipes & Tubing  9220X16X12   
802   Pipes & Tubing   9220X12X8   
803   Pipes & Tubing    9220X8X4   
804   Pipes & Tubing   9220X16X8   
805   Pipes & Tubing    9220X6X4   
806   Pipes & Tubing  9220X32X24   
807   Pipes & Tubing    9220X8X6   
808   Pipes & Tubing  9220X20X16   
809   Pipes & Tubing  9220X32X20   
810   Pipes & Tubing  9220X32X16   
811   Pipes & Tubing  9220X24X20   
812   Pipes & Tubing    9220X4X2   
813   Pipes & Tubing   9220X12X4   
814   Pipes & Tubing   9220X12X6   
815   Pipes & Tubing  9220X32X12   
816   Pipes & Tubing    9220X6X2   
817   Pipes & Tubing  9220X24X16   
818   Pipes & Tubing  9220X20X12   
819   Pipes & Tubing  9220X24X12   
820   Pipes & Tubing   9220X24X8   
821   Pipes & Tubing   9220X20X8   
822   Pipes & Tubing    3220X8X6   
823   Pipes & Tubing    3220X8X4   
824   Pipes & Tubing    3220X6X4   
825   Pipes & Tubing   3220X12X8   
826   Pipes & Tubing   3220X12X6   
827   Pipes & Tubing    3220X4X2   
828   Pipes & Tubing    3220X6X2   
829   Pipes & Tubing  3220X16X12   
830   Pipes & Tubing    3220X8X2   
831   Pipes & Tubing   3220X12X4   
832   Pipes & Tubing   3220X16X8   
833   Pipes & Tubing   3220X16X4   
834   Pipes & Tubing   3220X12X2   
835   Pipes & Tub

In [65]:
# df[(df['name'].astype(str)!='[]')&(df['matches'].astype(str)!='[]')][0:500]

In [66]:
# two=df[(df['matches'].astype(str)=='[]')]
# print(len(two[two['name'].astype(str)!='[]']))
# two=two[two['name'].astype(str)!='[]']
# two[curation_col]=two['name'].apply(lambda x:re.sub(r'(?i)P\.?S\.?I\.?','',str(x))).apply(lambda x: re.sub(r'.+\-\s?',"['",str(x))).apply(lambda x: re.sub(r'(?<!\,)000',',000',str(x))).apply(lambda x: re.sub(r"\['",'["',str(x))).apply(lambda x: re.sub(r"\s?'\]",'"]',str(x))).apply(lambda x: re.sub(r"\s?\s?\s?'\s?\,\s?'",'","',str(x))).apply(lambda x: re.sub(r'200"\,"125','125","200',str(x))).apply(lambda x: re.sub(r'\["1\,500"\,"100"\,"2,000 "\]','["100","1,500","2,000"]',str(x))).apply(lambda x: re.sub(r'\s?"\]','"]',str(x)))                                                    
# two[curation_col].explode().value_counts()
# matchtwo=two[['external_id',curation_col]]

In [35]:
# two=df[(df['matches'].astype(str)=='[]')]
# # two['name']=two['name'].apply(lambda x: re.sub(r'(?i)\(|x','',str(x))).apply(lambda x: re.sub(r"'",'"',str(x))).apply(lambda x: re.sub(r'\s?\,\s?',',',str(x))).apply(lambda x: re.sub(r'"\]','""]',str(x))).apply(lambda x:re.sub(r'\[""|\["','["',str(x)))                                                                  
# pat=r'(?i)((?i).{0,20}dia.{0,20})|()'
# two['name']=two['product_name'].apply(lambda x: re_extract(pat, str(x)))
# two['long']=two['long_desc'].apply(lambda x: re_extract(pat, str(x)))
# two[(two['name'].astype(str)!='[]')|(two['long'].astype(str)!='[]')]

# N/A

In [73]:
na=df[(df['name'].astype(str)=='[]')&(df[field].astype(str)=='nan')&(df['ld'].astype(str)=='[]')]
print(len(na))
na[curation_col]='n/a'
matchna=na[['external_id',curation_col]]
# print(len(na)+len(matchcustom)+len(matchtwo))
na

325


buckets external_id  \
0              Hoses      W28471   
1              Hoses      W28472   
2              Hoses      W28473   
3              Hoses      W28539   
4              Hoses      W28550   
5              Hoses      W28552   
6              Hoses      W28554   
7              Hoses      W28555   
8              Hoses      W86471   
9              Hoses      W86472   
10             Hoses      W86473   
11             Hoses      W86539   
12             Hoses      W86550   
13             Hoses      W86555   
14             Hoses      W86559   
31             Hoses       54219   
32             Hoses       54218   
33             Hoses       54217   
37             Hoses       54206   
38             Hoses       54205   
39             Hoses       54204   
40             Hoses       54203   
44             Hoses       54226   
45             Hoses       54208   
46             Hoses       54207   
47             Hoses       54224   
48             Hoses       54225   
49             Hoses       54223   
50             Hoses     5444950   
51             Hoses     5445050   
52             Hoses       54411   
53             Hoses     5445150   
54             Hoses     5445250   
55             Hoses     5445450   
56             Hoses       54454   
57             Hoses       54452   
58             Hoses       54451   
59             Hoses       54450   
60             Hoses       54424   
61             Hoses       54421   
62             Hoses       54412   
63             Hoses       54449   
64             Hoses       54422   
65             Hoses       54414   
66             Hoses       54441   
67             Hoses       54442   
68             Hoses       54444   
69             Hoses       54410   
70             Hoses       54440   
71             Hoses       54431   
72             Hoses       54432   
73             Hoses       54430   
498            Hoses       54631   
508            Hoses       54382   
509            Hoses       54383   
510            Hoses       54384   
511            Hoses       54385   
512            Hoses       54386   
513            Hoses       54387   
522            Hoses       54638   
524            Hoses       54637   
526            Hoses       54640   
527            Hoses       54633   
528            Hoses       54634   
529            Hoses       54639   
539            Hoses       54630   
540            Hoses       54632   
544            Hoses       54510   
545            Hoses       54512   
546            Hoses       54513   
547            Hoses       54514   
548            Hoses       54515   
549            Hoses       54516   
550            Hoses       54666   
551            Hoses       54662   
552            Hoses       54668   
554            Hoses       54671   
555            Hoses       54661   
556            Hoses       54665   
557            Hoses       54682   
558            Hoses       54667   
559            Hoses       54669   
561            Hoses       54670   
565            Hoses       54681   
566            Hoses       54683   
567            Hoses       54809   
568            Hoses       54680   
569            Hoses       54757   
570            Hoses       54780   
571            Hoses       54781   
572            Hoses       54770   
573            Hoses       54774   
574            Hoses       54766   
575            Hoses       54779   
576            Hoses       54817   
577            Hoses       54768   
578            Hoses       54762   
579            Hoses       54772   
580            Hoses       54820   
581            Hoses       54778   
582            Hoses       54758   
583            Hoses       54760   
584            Hoses       54764   
585            Hoses       54769   
586            Hoses       54763   
587            Hoses       54759   
588            Hoses       54771   
589            Hoses       54773   
590            Hoses       54816   
591            Hoses       54756   
592            Ho

In [68]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchcustom))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchna))
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchtwo))

# def get_df_name(df):
#     name =[x for x in gglobals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchfinal))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchex))

matchcustom
matchna


In [15]:
stop

NameError: name 'stop' is not defined

# send to the folder for upload

In [74]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchcustom) 

In [75]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchna) 

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchtwo) 